In [ ]:
import pandas as pd

dataset = pd.read_csv('data/supermarket_sales.csv')
dataset.head()

In [ ]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

In [3]:
file = client.files.create(
    file=open('data/supermarket_sales.csv',  'rb'),
    purpose='assistants'
)

In [6]:
assistant = client.beta.assistants.create(
    name='Financial Analyst',
    instructions="You are a financial analyst of a supermarket, providing insights on sales, expenses, profits, and customer demographics, based on the .csv file provided.",
    tools=[{'type': 'code_interpreter'}],
    tool_resources={
        'code_interpreter': {
            'file_ids': [file.id]
        }
    },
    model='gpt-4o'
)

In [7]:
thread = client.beta.threads.create()

In [13]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='what is the average rating of the supermarket? Check the .csv file for the latest data.'
)

In [14]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='The name of the user is Daniel and he is a premium user.'
)

In [15]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    print('Thinking...')
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

Thinking...
Thinking...


In [16]:
if run.status == 'completed':
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    print(messages)
else:
    print(run.status)
    print('Error occurred while running the thread.')

SyncCursorPage[Message](data=[Message(id='msg_3LryGbtNs5Xi6ik32dmJBALa', assistant_id='asst_fGZdOzHwpcM1Js44BGPcOSdK', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The average rating of the supermarket, based on the latest data from the CSV file, is approximately 6.97. This confirms our earlier calculation. If you have any further questions or need additional analysis, please let me know!'), type='text')], created_at=1726647135, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_FHzBdVO733Vh47jNG7CHmL9z', status=None, thread_id='thread_lRcCX7KKCY6vS6rxRWOtnATw'), Message(id='msg_WgxyYU3NWKsefqgDvGGJFtk0', assistant_id='asst_fGZdOzHwpcM1Js44BGPcOSdK', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I have verified the CSV file and the calculation of the average rating. The value calculated is based on the data present in the file yo

In [17]:
print(messages.data[0].content[0].text.value)

The average rating of the supermarket, based on the latest data from the CSV file, is approximately 6.97. This confirms our earlier calculation. If you have any further questions or need additional analysis, please let me know!
